<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Linear regression

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start with getting the data.

In [ ]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2024-08-20 22:02:19--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1 [following]
--2024-08-20 22:02:20--  https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc95133806368c7d7ff8ea9c6742.dl.dropboxusercontent.com/cd/0/inline/CZCbi4xVwipU9-4TWzl4OzIl0yrmiNAdFAPtANFUnhnq5T3w3DzXB6-d_TwOhWu7VaY6RNV3h48LHQYf4qMjnarEEsAC-XmodE4_9oUSC6H6FPeD_H4TWNHViHrdgUa-79M/file?dl=1# [following]
--2024-08-20 22:02:20--  https://uc95133806368c7d7ff8ea9c6742.dl.dropboxusercontent.com/cd/0/

In [ ]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`).

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [ ]:
import numpy as np

def msle(ys, ps):
    assert len(ys) == len(ps)

    return ((np.log(1 + ys) - np.log(1 + ps))**2).mean()

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [ ]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################

import pandas as pd

mieszkania = pd.read_csv('mieszkania.csv')
mieszkania_test = pd.read_csv('mieszkania_test.csv')

mean_price = mieszkania['cena'].mean()
prices = np.full(mieszkania['cena'].shape, mean_price)

print(mean_price)
print(msle(prices, mieszkania['cena']))

507919.49
0.3915253538257009


Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

We hant the derivative to be 0. Thus:

 $$\frac{d \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+c))^2}{dc} = 0 \implies n\log(1+c) = \sum_{i=1}^n \log(1+y_i) \implies
c = \exp \left(\frac{\sum_{i=1}^n \log(1+y_i)} {n} \right) - 1.$$

In [ ]:
#############################################
# TODO: Find this constant and compute msle #
#############################################
c = np.exp((np.log(mieszkania['cena']+1)).mean()) - 1

print(c)
print(msle(np.full(mieszkania['cena'].shape, c), mieszkania['cena']))

431435.4915823427
0.3648896122133589


Now, let's implement a standard linear regression model.

In [ ]:
##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################

def mse(ys, y_pred):
  assert len(ys) == len(y_pred)
  return ((ys - y_pred)**2).mean()

mieszkania = pd.get_dummies(mieszkania, columns=['dzielnica'], drop_first=True)
mieszkania_test = pd.get_dummies(mieszkania_test, columns=['dzielnica'], drop_first=True)

for col in mieszkania.select_dtypes(include='bool').columns:
    mieszkania[col] = mieszkania[col].astype(int)

for col in mieszkania_test.select_dtypes(include='bool').columns:
    mieszkania[col] = mieszkania[col].astype(int)

xs = np.array(mieszkania.loc[:, mieszkania.columns != 'cena'])
ys = np.array(mieszkania['cena'])
xs_test = np.array(mieszkania_test.loc[:, mieszkania_test.columns != 'cena'])
ys_test = np.array(mieszkania_test['cena'])


xs = xs.astype(np.float64)
ys = ys.astype(np.float64)
xs_test = xs_test.astype(np.float64)
ys_test = ys_test.astype(np.float64)

In [ ]:
weights = np.zeros(xs[0].shape)
bias = 0.0
lr = 1e-7
n_epochs = 10000

losses = [mse(ys, xs @ weights + bias)]

for epoch in range(n_epochs):
  y_pred = xs @ weights + bias

  # Gradient descent - derivative of MSE Loss function
  weights -= lr * 2 / len(xs) * (xs.T @ (y_pred - ys))
  bias -= lr * 2 / len(xs) * np.sum(y_pred - ys)

  loss = mse(ys, xs @ weights + bias)
  losses.append(loss)

  if epoch % 10 == 0:
    print(f'Epoch {epoch}: Loss = {loss:8.8f}')


print("MSE Loss: ", mse(ys_test, xs_test @ weights + bias))
print("MSLE Loss: ", msle(ys_test, xs_test @ weights + bias))

Epoch 0: Loss = 86345525111.92558289
Epoch 10: Loss = 72436458525.68092346
Epoch 20: Loss = 72101175214.01080322
Epoch 30: Loss = 71767842366.00590515
Epoch 40: Loss = 71436448634.89360046
Epoch 50: Loss = 71106982739.91296387
Epoch 60: Loss = 70779433465.92849731
Epoch 70: Loss = 70453789663.04855347
Epoch 80: Loss = 70130040246.24563599
Epoch 90: Loss = 69808174194.97909546
Epoch 100: Loss = 69488180552.82003784
Epoch 110: Loss = 69170048427.07821655
Epoch 120: Loss = 68853766988.43139648
Epoch 130: Loss = 68539325470.55660248
Epoch 140: Loss = 68226713169.76367188
Epoch 150: Loss = 67915919444.63085938
Epoch 160: Loss = 67606933715.64260864
Epoch 170: Loss = 67299745464.82941437
Epoch 180: Loss = 66994344235.40984344
Epoch 190: Loss = 66690719631.43437195
Epoch 200: Loss = 66388861317.43177795
Epoch 210: Loss = 66088759018.05709839
Epoch 220: Loss = 65790402517.74190521
Epoch 230: Loss = 65493781660.34661865
Epoch 240: Loss = 65198886348.81466675
Epoch 250: Loss = 64905706544.828865

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint:
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [ ]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################
weights1 = np.zeros(xs[0].shape)
bias1 = np.zeros(ys.shape)
lr = 1e-7
n_epochs = 10000

ys_log = np.log1p(ys).copy()


losses1 = [mse(ys, xs @ weights1 + bias1 )]

for epoch in range(n_epochs):
  y_pred = xs @ weights1 + bias1

  # Gradient descent - derivative of MSE Loss function
  weights1 -= lr * 2 / len(xs) * (xs.T @ (y_pred - ys_log))
  bias1 -= lr * 2 / len(xs) * (y_pred - ys_log)

  #It's msle loss for the data before log1p tansformation
  loss = mse(ys_log, xs @ weights1 + biases)
  losses1.append(loss)

  if epoch % 10 == 0:
    print(f'Epoch {epoch}: Loss = {loss:8.8f}')


print("MSLE Loss: ", msle(ys_test, np.expm1(xs_test @ weights1 + bias1)))

Epoch 0: Loss = 9.17519111
Epoch 10: Loss = 0.35097124
Epoch 20: Loss = 0.34947372
Epoch 30: Loss = 0.34798491
Epoch 40: Loss = 0.34650476
Epoch 50: Loss = 0.34503322
Epoch 60: Loss = 0.34357024
Epoch 70: Loss = 0.34211577
Epoch 80: Loss = 0.34066977
Epoch 90: Loss = 0.33923217
Epoch 100: Loss = 0.33780294
Epoch 110: Loss = 0.33638202
Epoch 120: Loss = 0.33496937
Epoch 130: Loss = 0.33356494
Epoch 140: Loss = 0.33216868
Epoch 150: Loss = 0.33078054
Epoch 160: Loss = 0.32940047
Epoch 170: Loss = 0.32802844
Epoch 180: Loss = 0.32666438
Epoch 190: Loss = 0.32530826
Epoch 200: Loss = 0.32396003
Epoch 210: Loss = 0.32261965
Epoch 220: Loss = 0.32128706
Epoch 230: Loss = 0.31996222
Epoch 240: Loss = 0.31864509
Epoch 250: Loss = 0.31733562
Epoch 260: Loss = 0.31603377
Epoch 270: Loss = 0.31473950
Epoch 280: Loss = 0.31345275
Epoch 290: Loss = 0.31217349
Epoch 300: Loss = 0.31090167
Epoch 310: Loss = 0.30963725
Epoch 320: Loss = 0.30838018
Epoch 330: Loss = 0.30713043
Epoch 340: Loss = 0.30588

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code.

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint:
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2:
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [ ]:
###############################################
# TODO: Implement the feature engineering part #
###############################################
from itertools import combinations

mieszkania = pd.read_csv('mieszkania.csv')
mieszkania_test = pd.read_csv('mieszkania_test.csv')

mean_price = mieszkania['cena'].mean()
prices = np.full(mieszkania['cena'].shape, mean_price)


df = pd.DataFrame(mieszkania['dzielnica'])
df['price_per_sq_meter'] = mieszkania['cena'] / mieszkania['m2']

avg_price_per_district = df.groupby('dzielnica')['price_per_sq_meter'].mean().reset_index()
avg_price_per_district.columns = ['dzielnica', 'avg_price_per_sq_meter']

mieszkania = pd.merge(mieszkania, avg_price_per_district, on='dzielnica', how='left')
mieszkania_test = pd.merge(mieszkania_test, avg_price_per_district, on='dzielnica', how='left')

mieszkania = pd.get_dummies(mieszkania, columns=['dzielnica'], drop_first=True)
mieszkania_test = pd.get_dummies(mieszkania_test, columns=['dzielnica'], drop_first=True)

for col in mieszkania.select_dtypes(include='bool').columns:
    mieszkania[col] = mieszkania[col].astype(int)

for col in mieszkania_test.select_dtypes(include='bool').columns:
    mieszkania[col] = mieszkania[col].astype(int)

df_feature_engineering = pd.DataFrame()
df_feature_engineering_test = pd.DataFrame()
columns = [col for col in mieszkania.columns if col != 'cena']

for col1, col2 in combinations(columns, 2):
  df_feature_engineering[f'{col1}_{col2}'] = mieszkania[col1] * mieszkania[col2]
  df_feature_engineering_test[f'{col1}_{col2}'] = mieszkania_test[col1] * mieszkania_test[col2]

df_feature_engineering['cena'] = mieszkania['cena']
df_feature_engineering_test['cena'] = mieszkania_test['cena']


xs = np.array(df_feature_engineering.loc[:, df_feature_engineering.columns != 'cena'])
ys = np.array(df_feature_engineering['cena'])
xs_test = np.array(df_feature_engineering_test.loc[:, df_feature_engineering_test.columns != 'cena'])
ys_test = np.array(df_feature_engineering_test['cena'])


xs = xs.astype(np.float64)
ys = ys.astype(np.float64)
xs_test = xs_test.astype(np.float64)
ys_test = ys_test.astype(np.float64)

In [ ]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################

weights = np.zeros(xs[0].shape)
bias = 0.0
lr = 1e-15
n_epochs = 50000

losses = [mse(ys, xs @ weights + bias)]

for epoch in range(n_epochs):
  y_pred = xs @ weights + bias

  # Gradient descent - derivative of MSE Loss function
  weights -= lr * 2 / len(xs) * (xs.T @ (y_pred - ys))
  bias -= lr * 2 / len(xs) * np.sum(y_pred - ys)

  loss = mse(ys, xs @ weights + bias)
  losses.append(loss)

  if epoch % 10 == 0:
    print(f'Epoch {epoch}: Loss = {loss:8.8f}')


print("MSLE Loss: ", msle(ys_test, xs_test @ weights + bias))
print("MSL Loss: ", mse(ys_test, xs_test @ weights + bias))

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Epoch 20: Loss = 59535250747.89872742
Epoch 30: Loss = 59371385708.55659485
Epoch 40: Loss = 59207995038.90726471
Epoch 50: Loss = 59045064296.29031372
Epoch 60: Loss = 58882592181.65605164
Epoch 70: Loss = 58720577403.91619110
Epoch 80: Loss = 58559018675.61837769
Epoch 90: Loss = 58397914712.93461609
Epoch 100: Loss = 58237264235.65100861
Epoch 110: Loss = 58077065967.15756226
Epoch 120: Loss = 57917318634.43814087
Epoch 130: Loss = 57758020968.06027985
Epoch 140: Loss = 57599171702.16510010
Epoch 150: Loss = 57440769574.45720673
Epoch 160: Loss = 57282813326.19479370
Epoch 170: Loss = 57125301702.17947388
Epoch 180: Loss = 56968233450.74644470
Epoch 190: Loss = 56811607323.75442505
Epoch 200: Loss = 56655422076.57582092
Epoch 210: Loss = 56499676468.08679962
Epoch 220: Loss = 56344369260.65740204
Epoch 230: Loss = 56189499220.14177704
Epoch 240: Loss = 56035065115.86830902
Epoch 250: Loss = 55881065720.62984467
Epoch 2